# Treinando RL para equilibrar o Cartpole

Este notebook faz parte do [Currículo de IA para Iniciantes](http://aka.ms/ai-beginners). Ele foi inspirado pelo [tutorial oficial do PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) e por [esta implementação de Cartpole em PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

Neste exemplo, usaremos RL para treinar um modelo a equilibrar um poste em um carrinho que pode se mover para a esquerda e para a direita em uma escala horizontal. Utilizaremos o ambiente [OpenAI Gym](https://www.gymlibrary.ml/) para simular o poste.

> **Nota**: Você pode executar o código desta lição localmente (por exemplo, no Visual Studio Code), caso em que a simulação será aberta em uma nova janela. Ao executar o código online, pode ser necessário fazer alguns ajustes no código, conforme descrito [aqui](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Começaremos garantindo que o Gym esteja instalado:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Agora vamos criar o ambiente CartPole e ver como operá-lo. Um ambiente possui as seguintes propriedades:

* **Action space** é o conjunto de ações possíveis que podemos realizar em cada etapa da simulação
* **Observation space** é o espaço de observações que podemos fazer


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Vamos ver como a simulação funciona. O loop a seguir executa a simulação até que `env.step` não retorne o sinal de término `done`. Escolheremos ações aleatoriamente usando `env.action_space.sample()`, o que significa que o experimento provavelmente falhará muito rápido (o ambiente CartPole termina quando a velocidade do CartPole, sua posição ou ângulo estão fora de certos limites).

> A simulação será aberta em uma nova janela. Você pode executar o código várias vezes e observar como ele se comporta.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Você pode notar que as observações contêm 4 números. São eles:
- Posição do carrinho
- Velocidade do carrinho
- Ângulo da haste
- Taxa de rotação da haste

`rew` é a recompensa que recebemos a cada passo. No ambiente CartPole, você recebe 1 ponto de recompensa por cada passo de simulação, e o objetivo é maximizar a recompensa total, ou seja, o tempo que o CartPole consegue se equilibrar sem cair.

Durante o aprendizado por reforço, nosso objetivo é treinar uma **política** $\pi$, que para cada estado $s$ nos dirá qual ação $a$ tomar, essencialmente $a = \pi(s)$.

Se você quiser uma solução probabilística, pode pensar na política como retornando um conjunto de probabilidades para cada ação, ou seja, $\pi(a|s)$ significaria a probabilidade de tomarmos a ação $a$ no estado $s$.

## Método de Gradiente de Política

No algoritmo mais simples de RL, chamado **Gradiente de Política**, treinaremos uma rede neural para prever a próxima ação.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Vamos treinar a rede executando muitos experimentos e atualizando nossa rede após cada execução. Vamos definir uma função que executará o experimento e retornará os resultados (o chamado **rastro**) - todos os estados, ações (e suas probabilidades recomendadas) e recompensas:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Você pode executar um episódio com a rede não treinada e observar que a recompensa total (também conhecida como duração do episódio) é muito baixa:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Um dos aspectos complicados do algoritmo de gradiente de política é usar **recompensas descontadas**. A ideia é que calculamos o vetor de recompensas totais em cada etapa do jogo e, durante esse processo, descontamos as recompensas iniciais usando algum coeficiente $gamma$. Também normalizamos o vetor resultante, porque o utilizaremos como peso para afetar nosso treinamento:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Agora vamos começar o treinamento! Executaremos 300 episódios, e em cada episódio faremos o seguinte:

1. Executar o experimento e coletar o rastreamento
1. Calcular a diferença (`gradients`) entre as ações realizadas e as probabilidades previstas. Quanto menor for a diferença, mais certeza teremos de que tomamos a ação correta.
1. Calcular as recompensas descontadas e multiplicar os gradientes pelas recompensas descontadas - isso garantirá que os passos com recompensas mais altas tenham maior impacto no resultado final do que aqueles com recompensas mais baixas.
1. As ações-alvo esperadas para nossa rede neural serão parcialmente derivadas das probabilidades previstas durante a execução e parcialmente dos gradientes calculados. Usaremos o parâmetro `alpha` para determinar em que medida os gradientes e recompensas serão levados em conta - isso é chamado de *taxa de aprendizado* do algoritmo de reforço.
1. Por fim, treinamos nossa rede com os estados e ações esperadas, e repetimos o processo.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Agora vamos executar o episódio com renderização para ver o resultado:


In [ ]:
_ = run_episode(render=True)

Espero que você possa ver que o poste agora consegue se equilibrar muito bem!

## Modelo Ator-Crítico

O modelo Ator-Crítico é um desenvolvimento adicional dos gradientes de política, no qual construímos uma rede neural para aprender tanto a política quanto as recompensas estimadas. A rede terá duas saídas (ou você pode vê-la como duas redes separadas):
* **Ator** recomendará a ação a ser tomada, fornecendo a distribuição de probabilidade do estado, como no modelo de gradiente de política.
* **Crítico** estimará qual seria a recompensa dessas ações. Ele retorna as recompensas totais estimadas no futuro para o estado dado.

Vamos definir um modelo assim:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Precisaríamos modificar ligeiramente nossas funções `discounted_rewards` e `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Agora vamos executar o loop principal de treinamento. Usaremos o processo manual de treinamento da rede, calculando funções de perda adequadas e atualizando os parâmetros da rede:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Conclusão

Vimos dois algoritmos de aprendizado por reforço (RL) nesta demonstração: o gradiente de política simples e o mais sofisticado ator-crítico. Você pode perceber que esses algoritmos operam com noções abstratas de estado, ação e recompensa - o que significa que podem ser aplicados a ambientes muito diferentes.

O aprendizado por reforço nos permite descobrir a melhor estratégia para resolver um problema apenas observando a recompensa final. O fato de não precisarmos de conjuntos de dados rotulados nos permite repetir simulações várias vezes para otimizar nossos modelos. No entanto, ainda existem muitos desafios no aprendizado por reforço, que você pode explorar caso decida se aprofundar mais nessa área fascinante da inteligência artificial.



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automatizadas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autoritativa. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
